## libraries

In [10]:
import torch
from torch import nn
from torchvision.datasets import MNIST
from torchvision import transforms
from torch.utils.data import TensorDataset, DataLoader, random_split
from torch import optim

## reading data

In [11]:
t = transforms.Compose([
    transforms.RandomRotation(10),
    transforms.RandomAffine(0, translate=(0.1, 0.1)),
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

train = MNIST(root='./data', train=True, transform=t, download=True)
test = MNIST(root='./data', train=False, transform=t, download=True)

x_train, y_train = train.data, train.targets
x_test, y_test = test.data, test.targets

## preprocessing

In [12]:
def preprocessing(x, y):
    x = x.type(torch.float32)
    x = (x - x.mean()) / x.std()
    x = x.unsqueeze(1)
    return x, y
x_train, y_train = preprocessing(x_train, y_train)
x_test, y_test = preprocessing(x_test, y_test)

## dataloader

In [13]:
train_set = TensorDataset(x_train, y_train)
train_set, valid_set = random_split(train_set, [50000, 10000])

test_set = TensorDataset(x_test, y_test)

train_loader = DataLoader(train_set, batch_size=256, shuffle=True)
valid_loader = DataLoader(valid_set, batch_size=256, shuffle=True)
test_loader = DataLoader(test_set, batch_size=1024, shuffle=False)

## model

In [14]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(num_features=32),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(num_features=64),
            nn.ReLU(),
            nn.AdaptiveMaxPool2d((14, 14))
        )
        self.layer3 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(num_features=128),
            nn.ReLU(),
            nn.AdaptiveAvgPool2d((7, 7))
        )
        self.layer4 = nn.Sequential(
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(num_features=256),
            nn.ReLU(),
            nn.AdaptiveAvgPool2d((7, 7))
        )
        self.layer5 = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=256 * 7 * 7, out_features=256),
            nn.ReLU(),
            nn.Dropout(0.5)
        )
        self.layer6 = nn.Sequential(
            nn.Linear(in_features=256, out_features=128),
            nn.ReLU()
        )
        self.layer7 = nn.Sequential(
            nn.Linear(in_features=128, out_features=10),
        )

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.layer5(out)
        out = self.layer6(out)
        out = self.layer7(out)
        return out

## hyper parameters

In [15]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = ConvNet().to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
epoch = 30
loss_thresh = 1

## train

In [16]:
model.train()
for epoch in range(epoch):
    for i, (image, label) in enumerate(train_loader):
        image = image.to(device)
        label = label.to(device)
        optimizer.zero_grad()
        output = model(image)
        loss = loss_fn(output, label)
        if loss.item() < loss_thresh:
            torch.save(model, 'model.ckpt')
            loss_thresh = loss.item()
        loss.backward()
        optimizer.step()
        if i % 100 == 0:
            print(f"epoch: {epoch}, loss: {loss.item()}")

epoch: 0, loss: 2.305349826812744
epoch: 0, loss: 0.0597643107175827
epoch: 1, loss: 0.04330001398921013
epoch: 1, loss: 0.07739507406949997
epoch: 2, loss: 0.026307400315999985
epoch: 2, loss: 0.0780431479215622
epoch: 3, loss: 0.03432378172874451
epoch: 3, loss: 0.03570227697491646
epoch: 4, loss: 0.016359742730855942
epoch: 4, loss: 0.03945133462548256
epoch: 5, loss: 0.0169031023979187
epoch: 5, loss: 0.020414823666214943
epoch: 6, loss: 0.007927187718451023
epoch: 6, loss: 0.013646220788359642
epoch: 7, loss: 0.01542949303984642
epoch: 7, loss: 0.07200226187705994
epoch: 8, loss: 0.05090324580669403
epoch: 8, loss: 0.018404094502329826
epoch: 9, loss: 0.002031586365774274
epoch: 9, loss: 0.03749333322048187
epoch: 10, loss: 0.037784479558467865
epoch: 10, loss: 0.021196449175477028
epoch: 11, loss: 0.017011458054184914
epoch: 11, loss: 0.0007282367441803217
epoch: 12, loss: 0.01117569487541914
epoch: 12, loss: 0.006575259380042553
epoch: 13, loss: 0.016123831272125244
epoch: 13, l

## validation

In [17]:
model.eval()
with torch.no_grad():
    for i, (image, label) in enumerate(valid_loader):
        image = image.to(device)
        label = label.to(device)
        output = model(image)
        loss = loss_fn(output, label)
        if i % 10 == 0:
            print(f"loss: {loss.item()}")

loss: 0.03072669357061386
loss: 0.09085695445537567
loss: 0.04872990399599075
loss: 0.0049851397052407265
